In [10]:
import torch
import torchvision
from torchvision import models
from torchvision import transforms
import torch.utils.data as data
import torchvision.datasets as datasets

In [2]:
dir(models)

['AlexNet',
 'DenseNet',
 'GoogLeNet',
 'GoogLeNetOutputs',
 'Inception3',
 'InceptionOutputs',
 'MNASNet',
 'MobileNetV2',
 'MobileNetV3',
 'ResNet',
 'ShuffleNetV2',
 'SqueezeNet',
 'VGG',
 '_GoogLeNetOutputs',
 '_InceptionOutputs',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_utils',
 'alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'detection',
 'googlenet',
 'inception',
 'inception_v3',
 'mnasnet',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet',
 'mobilenet_v2',
 'mobilenet_v3_large',
 'mobilenet_v3_small',
 'mobilenetv2',
 'mobilenetv3',
 'quantization',
 'resnet',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'resnext101_32x8d',
 'resnext50_32x4d',
 'segmentation',
 'shufflenet_v2_x0_5',
 'shufflenet_v2_x1_0',
 'shufflenet_v2_x1_5',
 'shufflenet_v2_x2_0',
 'shufflenetv2',
 'squeezenet',
 'squeezenet1_0',
 '

In [3]:
alexnet = models.alexnet(pretrained=True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


In [4]:
print(alexnet)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [5]:
transform = transforms.Compose([
                               transforms.Resize(256),
                               transforms.CenterCrop(244),
                               transforms.ToTensor(),
                               transforms.Normalize(
                                   mean = [0.485, 0.456, 0.406],
                                   std = [0.229, 0.224, 0.255]
                               )
])

In [6]:
validation_set = torchvision.datasets.ImageNet('./drive/MyDrive/Study/ImageNet', split='val', download=None, transform=transform)
# validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=64, shuffle=False)

In [11]:
validation_loader = data.DataLoader(validation_set, batch_size=100, shuffle=True, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [17]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = torchvision.models.alexnet(pretrained=True).to(device)
model.eval()

correct_top1 = 0
total = 0

with torch.no_grad():
        for idx, (images, labels) in enumerate(validation_loader):

            images = images.to(device)      # [100, 3, 224, 224]
            labels = labels.to(device)      # [100]
            outputs = model(images)

            # print(images.size()) # torch.Size([100, 3, 244, 244])
            # print(labels.size()) # torch.Size([100])
            # print(outputs.size()) # torch.Size([100, 1000])

            # ------------------------------------------------------------------------------
            # rank 1
            _, pred = torch.max(outputs, 1)
            total += labels.size(0)
            correct_top1 += (pred == labels).sum().item()

            print("step : {} / {}".format(idx + 1, len(validation_set)/int(labels.size(0))))
            print("top-1 percentage :  {0:0.2f}%".format(correct_top1 / total * 100))
            
print("top-1 percentage :  {0:0.2f}%".format(correct_top1 / total * 100))


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


step : 1 / 500.0
top-1 percentage :  52.00%
step : 2 / 500.0
top-1 percentage :  57.00%
step : 3 / 500.0
top-1 percentage :  55.33%
step : 4 / 500.0
top-1 percentage :  53.50%
step : 5 / 500.0
top-1 percentage :  52.60%
step : 6 / 500.0
top-1 percentage :  52.67%
step : 7 / 500.0
top-1 percentage :  52.57%
step : 8 / 500.0
top-1 percentage :  52.50%
step : 9 / 500.0
top-1 percentage :  52.22%
step : 10 / 500.0
top-1 percentage :  53.70%
step : 11 / 500.0
top-1 percentage :  53.82%
step : 12 / 500.0
top-1 percentage :  54.17%
step : 13 / 500.0
top-1 percentage :  53.92%
step : 14 / 500.0
top-1 percentage :  53.93%
step : 15 / 500.0
top-1 percentage :  53.80%
step : 16 / 500.0
top-1 percentage :  54.25%
step : 17 / 500.0
top-1 percentage :  54.53%
step : 18 / 500.0
top-1 percentage :  54.61%
step : 19 / 500.0
top-1 percentage :  54.89%
step : 20 / 500.0
top-1 percentage :  55.10%
step : 21 / 500.0
top-1 percentage :  55.24%
step : 22 / 500.0
top-1 percentage :  55.41%
step : 23 / 500.0
t

In [18]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = torchvision.models.vgg16(pretrained=True).to(device)
model.eval()

correct_top1 = 0
total = 0

with torch.no_grad():
        for idx, (images, labels) in enumerate(validation_loader):

            images = images.to(device)      # [100, 3, 224, 224]
            labels = labels.to(device)      # [100]
            outputs = model(images)

            # print(images.size()) # torch.Size([100, 3, 244, 244])
            # print(labels.size()) # torch.Size([100])
            # print(outputs.size()) # torch.Size([100, 1000])

            # ------------------------------------------------------------------------------
            # rank 1
            _, pred = torch.max(outputs, 1)
            total += labels.size(0)
            correct_top1 += (pred == labels).sum().item()

            print("step : {} / {}".format(idx + 1, len(validation_set)/int(labels.size(0))))
            print("top-1 percentage :  {0:0.2f}%".format(correct_top1 / total * 100))
            
print("top-1 percentage :  {0:0.2f}%".format(correct_top1 / total * 100))


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


step : 1 / 500.0
top-1 percentage :  75.00%
step : 2 / 500.0
top-1 percentage :  71.00%
step : 3 / 500.0
top-1 percentage :  65.67%
step : 4 / 500.0
top-1 percentage :  67.25%
step : 5 / 500.0
top-1 percentage :  68.40%
step : 6 / 500.0
top-1 percentage :  69.50%
step : 7 / 500.0
top-1 percentage :  68.86%
step : 8 / 500.0
top-1 percentage :  69.75%
step : 9 / 500.0
top-1 percentage :  70.33%
step : 10 / 500.0
top-1 percentage :  70.80%
step : 11 / 500.0
top-1 percentage :  70.82%
step : 12 / 500.0
top-1 percentage :  71.33%
step : 13 / 500.0
top-1 percentage :  70.77%
step : 14 / 500.0
top-1 percentage :  70.86%
step : 15 / 500.0
top-1 percentage :  71.00%
step : 16 / 500.0
top-1 percentage :  71.19%
step : 17 / 500.0
top-1 percentage :  71.41%
step : 18 / 500.0
top-1 percentage :  71.11%
step : 19 / 500.0
top-1 percentage :  71.21%
step : 20 / 500.0
top-1 percentage :  71.35%
step : 21 / 500.0
top-1 percentage :  71.29%
step : 22 / 500.0
top-1 percentage :  71.23%
step : 23 / 500.0
t

In [19]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = torchvision.models.resnet18(pretrained=True).to(device)
model.eval()

correct_top1 = 0
total = 0

with torch.no_grad():
        for idx, (images, labels) in enumerate(validation_loader):

            images = images.to(device)      # [100, 3, 224, 224]
            labels = labels.to(device)      # [100]
            outputs = model(images)

            # print(images.size()) # torch.Size([100, 3, 244, 244])
            # print(labels.size()) # torch.Size([100])
            # print(outputs.size()) # torch.Size([100, 1000])

            # ------------------------------------------------------------------------------
            # rank 1
            _, pred = torch.max(outputs, 1)
            total += labels.size(0)
            correct_top1 += (pred == labels).sum().item()

            print("step : {} / {}".format(idx + 1, len(validation_set)/int(labels.size(0))))
            print("top-1 percentage :  {0:0.2f}%".format(correct_top1 / total * 100))
            
print("top-1 percentage :  {0:0.2f}%".format(correct_top1 / total * 100))


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


step : 1 / 500.0
top-1 percentage :  77.00%
step : 2 / 500.0
top-1 percentage :  74.50%
step : 3 / 500.0
top-1 percentage :  71.67%
step : 4 / 500.0
top-1 percentage :  72.50%
step : 5 / 500.0
top-1 percentage :  71.60%
step : 6 / 500.0
top-1 percentage :  72.67%
step : 7 / 500.0
top-1 percentage :  72.57%
step : 8 / 500.0
top-1 percentage :  72.88%
step : 9 / 500.0
top-1 percentage :  72.44%
step : 10 / 500.0
top-1 percentage :  70.70%
step : 11 / 500.0
top-1 percentage :  70.18%
step : 12 / 500.0
top-1 percentage :  70.42%
step : 13 / 500.0
top-1 percentage :  70.69%
step : 14 / 500.0
top-1 percentage :  70.79%
step : 15 / 500.0
top-1 percentage :  70.80%
step : 16 / 500.0
top-1 percentage :  71.38%
step : 17 / 500.0
top-1 percentage :  71.35%
step : 18 / 500.0
top-1 percentage :  71.67%
step : 19 / 500.0
top-1 percentage :  71.63%
step : 20 / 500.0
top-1 percentage :  71.40%
step : 21 / 500.0
top-1 percentage :  71.52%
step : 22 / 500.0
top-1 percentage :  71.36%
step : 23 / 500.0
t

In [20]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = torchvision.models.googlenet(pretrained=True).to(device)
model.eval()

correct_top1 = 0
total = 0

with torch.no_grad():
        for idx, (images, labels) in enumerate(validation_loader):

            images = images.to(device)      # [100, 3, 224, 224]
            labels = labels.to(device)      # [100]
            outputs = model(images)

            # print(images.size()) # torch.Size([100, 3, 244, 244])
            # print(labels.size()) # torch.Size([100])
            # print(outputs.size()) # torch.Size([100, 1000])

            # ------------------------------------------------------------------------------
            # rank 1
            _, pred = torch.max(outputs, 1)
            total += labels.size(0)
            correct_top1 += (pred == labels).sum().item()

            print("step : {} / {}".format(idx + 1, len(validation_set)/int(labels.size(0))))
            print("top-1 percentage :  {0:0.2f}%".format(correct_top1 / total * 100))
            
print("top-1 percentage :  {0:0.2f}%".format(correct_top1 / total * 100))


Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


step : 1 / 500.0
top-1 percentage :  72.00%
step : 2 / 500.0
top-1 percentage :  69.00%
step : 3 / 500.0
top-1 percentage :  67.00%
step : 4 / 500.0
top-1 percentage :  69.25%
step : 5 / 500.0
top-1 percentage :  71.80%
step : 6 / 500.0
top-1 percentage :  71.00%
step : 7 / 500.0
top-1 percentage :  70.57%
step : 8 / 500.0
top-1 percentage :  70.25%
step : 9 / 500.0
top-1 percentage :  70.44%
step : 10 / 500.0
top-1 percentage :  70.40%
step : 11 / 500.0
top-1 percentage :  70.09%
step : 12 / 500.0
top-1 percentage :  70.67%
step : 13 / 500.0
top-1 percentage :  70.77%
step : 14 / 500.0
top-1 percentage :  70.79%
step : 15 / 500.0
top-1 percentage :  70.60%
step : 16 / 500.0
top-1 percentage :  70.31%
step : 17 / 500.0
top-1 percentage :  70.06%
step : 18 / 500.0
top-1 percentage :  70.06%
step : 19 / 500.0
top-1 percentage :  69.89%
step : 20 / 500.0
top-1 percentage :  70.05%
step : 21 / 500.0
top-1 percentage :  70.00%
step : 22 / 500.0
top-1 percentage :  70.18%
step : 23 / 500.0
t